In [1]:
import requests
import json
import xml.etree.ElementTree as ET
from time import gmtime, strftime, sleep
import time
import datetime
import boto3
from multiprocessing import Pool

In [2]:
kraje = {
    'CZ010': 'Hlavní město Praha',
    'CZ020': 'Středočeský kraj',
    'CZ031': 'Jihočeský kraj',
    'CZ032': 'Plzeňský kraj',
    'CZ041': 'Karlovarský kraj',
    'CZ042': 'Ústecký kraj',
    'CZ051': 'Liberecký kraj',
    'CZ052': 'Královéhradecký kraj',
    'CZ053': 'Pardubický kraj',
    'CZ063': 'Kraj Vysočina',
    'CZ064': 'Jihomoravský kraj',
    'CZ071': 'Olomoucký kraj',
    'CZ072': 'Zlínský kraj',
    'CZ080': 'Moravskoslezský kraj'
}

In [3]:
kandidati = {
    '1': 'Mirek Topolánek',
    '2': 'Michal Horáček',
    '3': 'Pavel Fischer',
    '4': 'Jiří Hynek',
    '5': 'Petr Hannig',
    '6': 'Vratislav Kulhánek',
    '7': 'Miloš Zeman',
    '8': 'Marek Hilšer',
    '9': 'Jiří Drahoš'
}

In [4]:
ns = '{http://www.volby.cz/prezident/}'
s3 = boto3.client('s3')

In [7]:
def write(out):
    obid = out['CIS_OBEC']
    print(obid + ' | ' + strftime("%Y-%m-%d %H_%M_%S", time.localtime())) 
    putFile = s3.put_object(Bucket='prezident17', 
                            Key='kolo1/obce/' + obid + '.json', 
                            Body=json.dumps(out), 
                            ACL='public-read', 
                            ContentType='application/json')

while True:
    for kraj in kraje:
        print(kraje[kraj])
        kr_data = []
        now = str(datetime.datetime.utcnow().isoformat()) + 'Z'
        r = requests.get('http://80.188.53.60/pls/prezmedia/vysledky_kraj?kolo=&nuts=' + kraj)
        root = ET.fromstring(r.text)
        for okres in root[0].findall(ns + 'OKRES'):    
            for obec in okres.findall(ns + 'OBEC'):
                out = {
                    'UPDATED': now,
                    'OKRES': okres.attrib['NUTS_OKRES'],
                    'NAZ_OKRES': okres.attrib['NAZ_OKRES'],
                    'KAND': {}
                }
                out.update(obec.attrib)
                out.update(obec.find(ns + 'UCAST').attrib)
                
                for kn in obec.findall(ns + 'HODN_KAND'):
                    kn_id = kn.attrib['PORADOVE_CISLO']

                    out['KAND'][kn_id] = kn.attrib
                    out['KAND'][kn_id].update({'JMENO': kandidati[kn.attrib['PORADOVE_CISLO']]})                    
                
                kr_data.append(out)

        with Pool(150) as p:
            p.map(write, kr_data)
        
    break #remove before flight

Hlavní město Praha
{'UPDATED': '2017-12-19T14:50:06.865765Z', 'OKRES': 'CZ010', 'NAZ_OKRES': 'Hlavní město Praha', 'KAND': {'1': {'PORADOVE_CISLO': '1', 'HLASY': '5143', 'JMENO': 'Mirek Topolánek'}, '2': {'PORADOVE_CISLO': '2', 'HLASY': '4676', 'JMENO': 'Michal Horáček'}, '3': {'PORADOVE_CISLO': '3', 'HLASY': '4998', 'JMENO': 'Pavel Fischer'}, '4': {'PORADOVE_CISLO': '4', 'HLASY': '7728', 'JMENO': 'Jiří Hynek'}, '5': {'PORADOVE_CISLO': '5', 'HLASY': '5497', 'JMENO': 'Petr Hannig'}, '6': {'PORADOVE_CISLO': '6', 'HLASY': '5508', 'JMENO': 'Vratislav Kulhánek'}, '7': {'PORADOVE_CISLO': '7', 'HLASY': '5436', 'JMENO': 'Miloš Zeman'}, '8': {'PORADOVE_CISLO': '8', 'HLASY': '8312', 'JMENO': 'Marek Hilšer'}, '9': {'PORADOVE_CISLO': '9', 'HLASY': '5085', 'JMENO': 'Jiří Drahoš'}}, 'CIS_OBEC': '500178', 'NAZ_OBEC': 'Praha 6', 'TYP_OBEC': 'MCMO', 'KOLO': '1', 'OKRSKY_CELKEM': '104', 'OKRSKY_ZPRAC': '104', 'OKRSKY_ZPRAC_PROC': '100.00', 'ZAPSANI_VOLICI': '63907', 'VYDANE_OBALKY': '55008', 'UCAST_PROC

{'UPDATED': '2017-12-19T14:50:08.584918Z', 'OKRES': 'CZ0631', 'NAZ_OKRES': 'Havlíčkův Brod', 'KAND': {'1': {'PORADOVE_CISLO': '1', 'HLASY': '37', 'JMENO': 'Mirek Topolánek'}, '2': {'PORADOVE_CISLO': '2', 'HLASY': '15', 'JMENO': 'Michal Horáček'}, '3': {'PORADOVE_CISLO': '3', 'HLASY': '75', 'JMENO': 'Pavel Fischer'}, '4': {'PORADOVE_CISLO': '4', 'HLASY': '144', 'JMENO': 'Jiří Hynek'}, '5': {'PORADOVE_CISLO': '5', 'HLASY': '11', 'JMENO': 'Petr Hannig'}, '6': {'PORADOVE_CISLO': '6', 'HLASY': '45', 'JMENO': 'Vratislav Kulhánek'}, '7': {'PORADOVE_CISLO': '7', 'HLASY': '73', 'JMENO': 'Miloš Zeman'}, '8': {'PORADOVE_CISLO': '8', 'HLASY': '71', 'JMENO': 'Marek Hilšer'}, '9': {'PORADOVE_CISLO': '9', 'HLASY': '24', 'JMENO': 'Jiří Drahoš'}}, 'CIS_OBEC': '548189', 'NAZ_OBEC': 'Kouty', 'TYP_OBEC': 'OBEC_BEZ_MCMO', 'KOLO': '1', 'OKRSKY_CELKEM': '1', 'OKRSKY_ZPRAC': '1', 'OKRSKY_ZPRAC_PROC': '100.00', 'ZAPSANI_VOLICI': '604', 'VYDANE_OBALKY': '520', 'UCAST_PROC': '86.09', 'ODEVZDANE_OBALKY': '510', '